In [384]:
import numpy as np
from scipy.fftpack import fft, ifft, fftfreq, hilbert
from scipy import signal
from scipy.signal import hilbert as analytic
import matplotlib.pyplot as plt
import mpld3
from mpld3 import plugins
#mpl.style.use('default')
#mpl.rcParams['text.color'] = 'w'

In [399]:
fc = 40e3
f0 = 1e3
fs = 200e3

db = lambda x: 20*np.log10(np.abs(x))
periods = 10
nsamples = np.ceil(periods *fs/f0).astype('int')
t=np.arange(nsamples)/fs
f = fftfreq(nsamples,1/fs)
carrier = np.exp(1j*2*np.pi*fc*t)
audio = np.sin(2*np.pi*f0*t)#+0.7*np.sin(4*np.pi*f0*t)+0.5*np.sin(6*np.pi*f0*t)+0.3*np.sin(8*np.pi*f0*t)+0.1*np.sin(10*np.pi*f0*t)
audio /= np.max(np.abs(audio))

The square root in the envelope calculation will introduce an infinite number of harmonics in the envelope spectrum. 
If we modulate the carriwe with the bare envelope we will introcduce a lot of harmonics in the signal. This cannot be reproduced in a trancduser with a small bandwidth.

In [400]:
envelope = np.sqrt(1+0.95*audio)
envelope_spectrum = fft(envelope)
modulated = envelope*carrier.real
modulated_spectrum = fft(modulated)

fig, ax = plt.subplots(1,2)
ax[0].plot(f,db(envelope_spectrum))
ax[1].plot(f,db(modulated_spectrum))
mpld3.display()

/Users/carl/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:5: RuntimeWarning: divide by zero encountered in log10


If we create the analytic signal of the envelope as 
$$m_a(t) = m(t) + j\mathcal{H}\{m(t)\} $$
we will obtain only sidebands on one side of the carrier.

In [402]:
analytic = envelope + 1j*hilbert(envelope)
modulated = np.real(analytic * carrier)
analytic_spectrum = fft(analytic)
modulated_spectrum = fft(modulated)

fig, ax = plt.subplots(1,2)
ax[0].plot(f,db(analytic_spectrum))
ax[1].plot(f,db(modulated_spectrum))
mpld3.display()

If we instead study the case when the modulation has been moved to the exponent, as
$$ y(t) = e^{\log(m(t))} e^{j\omega t} $$
we cad add a pure phase shift as 
$$ y(t) = e^{\log(m(t))} e^{j\phi(t)} e^{j\omega t} $$
If the phase shift $\phi$ is chosen to create an analytic signal, i.e.
$$ \phi(t) = \mathcal{H}\{\log(m(t))\} $$
we see that the new modulation is given by
$$ \tilde m = e^{ \log(m) +j\mathcal{H}\{\log(m)\} }  = m(t) e^{j\mathcal{H}\{\log(m(t))\} } $$
This technique was first introduced by Powers (1960) and explaned in an overview by Bedrosian (1962) as exponential or envelope modulation.
If we define $g(t) = \alpha \log f(t) $ where $f(t)$ is the nonnegative signal 1+audio, and modulate with the factor 
$$m(t) = e^{g(t)+j\mathcal{H}\{g(t)\}} $$
it can be showh that the envelope of the carrier (magnitude of the analytic carrier) will be proportional to $f^\alpha$. Since we want an envelope proportional to the square root of the audio signal, we choose $\alpha = 1/2$. This formalism reqires that the carrier is represented in the complex (analytic) domain.

In [405]:
logsignal = np.log(1+0.95*audio)/2
explog = np.exp(logsignal + 1j*hilbert(logsignal))
modulated = explog*carrier
explog_spectrum = fft(explog)
modulated_spectrum = fft(modulated)
fig, ax = plt.subplots(1,2)
ax[0].plot(f,db(explog_spectrum))
ax[1].plot(f,db(modulated_spectrum))
mpld3.display()

In [366]:
%qtconsole